<a href="https://colab.research.google.com/github/nannthd/project_AIEngineer/blob/main/Fine_Tune_ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Fine-Tune ViT**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/drug/data 50 class add_augment3.zip'

In [ ]:
import os
import pandas as pd

# Path to your directory containing subfolders
img_dir = '/content/data 50 class add_augment'  # Adjust this path to your dataset

# Initialize an empty list to store folder names
folder_names = []

# Check if the main directory exists
if not os.path.exists(img_dir):
    print(f"Directory not found: {img_dir}")
else:
    # Loop through each subfolder in the image directory
    for folder_name in os.listdir(img_dir):
        folder_path = os.path.join(img_dir, folder_name)
        if os.path.isdir(folder_path):
            print(f"Found folder: {folder_name}")  # Debug print
            folder_names.append(folder_name)

# Create a DataFrame
if folder_names:
    df = pd.DataFrame(folder_names, columns=["class_name"])
    # Save the DataFrame to a CSV file
    labels_file = '/content/labels.csv'  # Adjust this path to where you want to save folder_names.csv
    df.to_csv(labels_file, index=False)
    print(f"Folder names CSV file saved to {labels_file}")
    # Print the DataFrame to check the collected data
    print(df.head())
else:
    print("No subfolders found. Please check the directory structure.")

In [ ]:
import os
import pandas as pd
from transformers import ViTForImageClassification, TrainingArguments, Trainer, default_data_collator
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from PIL import Image
import torch
from torch.utils.data import Dataset

# สร้างไฟล์ labels.csv
def create_labels_csv(img_dir, labels_file):
    img_paths = []
    labels = []

    # เดินไปยังโฟลเดอร์หลัก
    for folder_name in os.listdir(img_dir):
        folder_path = os.path.join(img_dir, folder_name)

        if os.path.isdir(folder_path):
            # เดินไปยังไฟล์ภาพในโฟลเดอร์ย่อย
            for img_name in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_name)
                img_paths.append(img_path)
                labels.append(folder_name)

    # สร้าง DataFrame และบันทึกเป็น CSV
    df = pd.DataFrame({
        'image_path': img_paths,
        'class_name': labels
    })
    df.to_csv(labels_file, index=False)

# กำหนดพาธ
img_dir = '/content/data 50 class add_augment'
labels_file = '/content/labels.csv'

# สร้างไฟล์ labels.csv
create_labels_csv(img_dir, labels_file)

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.img_labels = pd.read_csv(labels_file)
        self.label_map = {name: idx for idx, name in enumerate(self.img_labels['class_name'].unique())}

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_name = self.img_labels.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        label_name = self.img_labels.iloc[idx, 1]
        label = self.label_map[label_name]  # Convert class name to integer label

        if self.transform:
            image = self.transform(image)

        return {'pixel_values': image, 'labels': torch.tensor(label, dtype=torch.long)}

# Define transforms
transform = Compose([
    Resize((224, 224)),  # Use the size expected by ViT model
    CenterCrop(224),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create dataset
dataset = CustomDataset(img_dir=img_dir, labels_file=labels_file, transform=transform)

# Load Pre-trained Model และ Feature Extractor
model_name = "google/vit-base-patch16-224-in21k"
NUM_CLASSES = len(pd.read_csv(labels_file)['class_name'].unique())  # Number of unique classes
model = ViTForImageClassification.from_pretrained(model_name, num_labels=NUM_CLASSES)

# Check if GPU is available and move model to GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define Data Collator
data_collator = default_data_collator

# Define Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="steps",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    save_strategy="steps",
    eval_steps=1500,
    save_steps=1500,
    logging_dir='./logs',
    report_to="tensorboard",  # Optional: report to TensorBoard
)

# Define Trainer และ Train Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    data_collator=data_collator,
    tokenizer=None,  # Set to None if not using a tokenizer
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

In [6]:
# โหลดโมเดล
# Export Data drug50cls
# Define the folder you want to zip and download
import shutil

folder_to_download = '/content/results/checkpoint-3000'

# Zip the folder
shutil.make_archive('/content/drive/MyDrive/drug/FineTune/model_VIT_224', 'zip', folder_to_download)

# # Download the zip file
# from google.colab import files
# files.download('/content/drive/MyDrive/drug/FineTune/model_VIT_224.zip')

'/content/drive/MyDrive/drug/FineTune/model_VIT_224.zip'

##224*224

###50cls

In [ ]:
# use VIT model to convert data
import pandas as pd
import numpy as np
import os
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
# โหลดโมเดล ViT และ Processor
model_path = '/content/results/checkpoint-3000'  # เปลี่ยนเป็น path ที่ถูกต้อง
model = ViTModel.from_pretrained(model_path)  # ใช้ ViTModel แทน ViTForImageClassification
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

def image_embedding(path):
    img = Image.open(path).convert('RGB').resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ViT
    inputs = processor(images=img, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    avg_embedding = last_hidden_states.mean(dim=1).squeeze().detach().numpy()  # ค่าเฉลี่ยของ hidden states

    curr_df = pd.DataFrame(avg_embedding).T
    return curr_df

def process_images_in_folder(folder_path, output_csv_path):
    pdEmbedded = pd.DataFrame()
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    for idx, image_file in enumerate(image_files):
        image_path = os.path.join(folder_path, image_file)
        embedded = image_embedding(image_path)
        embedded['ID'] = image_file  # ใช้ชื่อไฟล์เป็น ID
        pdEmbedded = pd.concat([pdEmbedded, embedded], ignore_index=True)

    # สร้างโฟลเดอร์สำหรับ output_csv_path
    output_folder = os.path.dirname(output_csv_path)
    os.makedirs(output_folder, exist_ok=True)

    # บันทึก DataFrame ลงในไฟล์ CSV
    pdEmbedded.to_csv(output_csv_path, index=False)

# ฟังก์ชันสำหรับการวนลูปโฟลเดอร์
def process_all_folders(base_folder_path, output_base_folder):
    subfolders = [f.path for f in os.scandir(base_folder_path) if f.is_dir()]

    for subfolder in subfolders:
        subfolder_name = os.path.basename(subfolder)
        output_csv_path = os.path.join(output_base_folder, f"{subfolder_name}.csv")
        process_images_in_folder(subfolder, output_csv_path)
        print(f"Embedding data for {subfolder_name} has been saved to {output_csv_path}")

# ตัวอย่างการใช้งาน
base_folder_path = "/content/data 50 class add_augment"
output_base_folder = "/content/Vector_VIT_FT224"
os.makedirs(output_base_folder, exist_ok=True)  # สร้างโฟลเดอร์หลักถ้ายังไม่มี

process_all_folders(base_folder_path, output_base_folder)

In [8]:
# โหลดข้อมูลเวกเตอร์224*224
# Export Data drug50cls
# Define the folder you want to zip and download
import shutil

folder_to_download = '/content/Vector_VIT_FT224'

# Zip the folder
shutil.make_archive('/content/drive/MyDrive/drug/FineTune/Vector_VIT_FT224', 'zip', folder_to_download)

# # Download the zip file
# from google.colab import files
# files.download('/content/drive/MyDrive/drug/FineTune/Vector_VIT_FT224.zip')

'/content/drive/MyDrive/drug/FineTune/Vector_VIT_FT224.zip'

###Cosine similarity

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('/content/results/checkpoint-3000')

# ย้ายโมเดลไปยัง GPU ถ้ามี
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def image_embedding(path):
    try:
        img = Image.open(path).convert('RGB').resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ViT
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        avg_embedding = last_hidden_states.mean(dim=1).squeeze().cpu().detach().numpy()  # ค่าเฉลี่ยของ hidden states
        curr_df = pd.DataFrame(avg_embedding).T
        return curr_df
    except Exception as e:
        print(f"Error processing image {path}: {e}")
        return pd.DataFrame()

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/Vector_VIT_FT224'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# รูปภาพที่ต้องการทดสอบ
new_image_path = '/content/1.jpg'

# ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
similar_classes = find_most_similar_classes(new_image_path, all_embeddings)

# แสดงผลลัพธ์
print("Top 5 most similar classes:")
for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
    print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

Some weights of ViTModel were not initialized from the model checkpoint at /content/results/checkpoint-1250 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Top 5 most similar classes:
1. Class: Amlopine5mg, Similarity Score: 0.7520281472894522
2. Class: Amlopine10mg, Similarity Score: 0.38829910199583983
3. Class: Prenolol25mg, Similarity Score: 0.3289244473260224
4. Class: Prenolol100mg, Similarity Score: 0.2969507488183878
5. Class: Janumet50_1000mg, Similarity Score: 0.2915699594577877


#640*640

###50cls

In [ ]:
# use VIT model to convert data
import pandas as pd
import numpy as np
import os
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

def image_embedding(path):
    img = Image.open(path).convert('RGB').resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ViT
    inputs = processor(images=img, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    avg_embedding = last_hidden_states.mean(dim=1).squeeze().detach().numpy()  # ค่าเฉลี่ยของ hidden states

    curr_df = pd.DataFrame(avg_embedding).T
    return curr_df

def process_images_in_folder(folder_path, output_csv_path):
    pdEmbedded = pd.DataFrame()
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    for idx, image_file in enumerate(image_files):
        image_path = os.path.join(folder_path, image_file)
        embedded = image_embedding(image_path)
        embedded['ID'] = image_file  # ใช้ชื่อไฟล์เป็น ID
        pdEmbedded = pd.concat([pdEmbedded, embedded], ignore_index=True)

    # สร้างโฟลเดอร์สำหรับ output_csv_path
    output_folder = os.path.dirname(output_csv_path)
    os.makedirs(output_folder, exist_ok=True)

    # บันทึก DataFrame ลงในไฟล์ CSV
    pdEmbedded.to_csv(output_csv_path, index=False)

# ฟังก์ชันสำหรับการวนลูปโฟลเดอร์
def process_all_folders(base_folder_path, output_base_folder):
    subfolders = [f.path for f in os.scandir(base_folder_path) if f.is_dir()]

    for subfolder in subfolders:
        subfolder_name = os.path.basename(subfolder)
        output_csv_path = os.path.join(output_base_folder, f"{subfolder_name}.csv")
        process_images_in_folder(subfolder, output_csv_path)
        print(f"Embedding data for {subfolder_name} has been saved to {output_csv_path}")

# ตัวอย่างการใช้งาน
base_folder_path = "/content/data 50 class add_augment"
output_base_folder = "/content/Vector_VIT_FT640"
os.makedirs(output_base_folder, exist_ok=True)  # สร้างโฟลเดอร์หลักถ้ายังไม่มี

process_all_folders(base_folder_path, output_base_folder)

In [5]:
# โหลดข้อมูลเวกเต640*640
# Export Data drug50cls
# Define the folder you want to zip and download
import shutil

folder_to_download = '/content/Vector_VIT_FT640'

# Zip the folder
shutil.make_archive('/content/drive/MyDrive/drug/FineTune/Vector_VIT_FT640', 'zip', folder_to_download)

# # Download the zip file
# from google.colab import files
# files.download('/content/drive/MyDrive/drug/FineTune/Vector_VIT_FT640.zip')

'/content/drive/MyDrive/drug/FineTune/Vector_VIT_FT640.zip'

###Cosine similarity

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# ย้ายโมเดลไปยัง GPU ถ้ามี
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def image_embedding(path):
    try:
        img = Image.open(path).convert('RGB').resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ViT
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        avg_embedding = last_hidden_states.mean(dim=1).squeeze().cpu().detach().numpy()  # ค่าเฉลี่ยของ hidden states
        curr_df = pd.DataFrame(avg_embedding).T
        return curr_df
    except Exception as e:
        print(f"Error processing image {path}: {e}")
        return pd.DataFrame()

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls_ViTModel_640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# รูปภาพที่ต้องการทดสอบ
new_image_path = '/content/2.jpg'

# ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
similar_classes = find_most_similar_classes(new_image_path, all_embeddings)

# แสดงผลลัพธ์
print("Top 5 most similar classes:")
for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
    print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")